# Plan for data processing

In [1]:
# reading data in 
# for every subject, read in both data frames by processing each file 
# once all data frames read in, multiply sp02HR entries by 8 each
# merge data frames together 
# add column for physiological stress binary variable 
# DATA PROCESSING STEPS FROM ML CLASS
    # 60% of dataframes as training data, 20% as validation, 20% as testing
    # unbalanced data....? SMOTE?



# Libraries

In [63]:
import wfdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random

In [3]:
print(os.listdir())

['Subject9_SpO2HR.dat', 'Subject11_SpO2HR.dat', 'Subject12_SpO2HR.hea', 'Subject18_AccTempEDA.dat', 'Subject10_AccTempEDA.dat', 'Subject2_AccTempEDA.dat', 'Subject7_SpO2HR.hea', 'Subject4_SpO2HR.dat', 'Main.ipynb', 'Subject17_AccTempEDA.dat', 'RECORDS', 'Subject5_AccTempEDA.dat', 'Subject16_AccTempEDA.dat', 'Subject19_SpO2HR.dat', 'Subject1_SpO2HR.dat', 'Subject2_SpO2HR.hea', 'Subject4_AccTempEDA.dat', 'Subject11_AccTempEDA.dat', 'Subject20_AccTempEDA.atr', 'Subject17_SpO2HR.hea', 'Subject14_SpO2HR.dat', 'Subject20_SpO2HR.hea', 'Subject3_AccTempEDA.dat', 'subjectinfo.csv', 'Subject19_AccTempEDA.dat', 'Subject1_AccTempEDA.dat', 'Subject13_AccTempEDA.dat', 'Subject20_AccTempEDA.hea', 'Subject8_SpO2HR.hea', 'Subject10_SpO2HR.hea', 'Subject13_SpO2HR.dat', 'Subject9_AccTempEDA.dat', 'ANNOTATORS', 'Subject6_AccTempEDA.dat', 'Subject14_AccTempEDA.dat', 'Subject6_SpO2HR.dat', 'Subject5_SpO2HR.hea', 'Subject7_AccTempEDA.dat', 'Subject18_SpO2HR.hea', 'Subject3_SpO2HR.dat', 'Subject15_AccTempEDA.

# data processing

In [22]:


for file in os.listdir():
    if file.endswith(".dat"):
        record = wfdb.rdrecord(file[:-4], sampfrom=0, sampto=None)
        signals = record.p_signal
        raw_signals = record.d_signal
        signal_names = record.sig_name
        df = pd.DataFrame(data = signals, columns=record.sig_name)
        df['time'] = np.arange(len(df)) / record.fs
        # print(df.head())
        df.to_csv(file[:-4] + ".csv", index=False)



In [27]:
os.listdir()

['Subject9_SpO2HR.dat',
 'Subject11_SpO2HR.dat',
 'Subject12_SpO2HR.hea',
 'Subject18_AccTempEDA.dat',
 'Subject10_AccTempEDA.dat',
 'Subject7_SpO2HR.csv',
 'Subject2_AccTempEDA.dat',
 'Subject7_SpO2HR.hea',
 'Subject4_SpO2HR.dat',
 'Main.ipynb',
 'Subject17_AccTempEDA.dat',
 'Subject12_SpO2HR.csv',
 'RECORDS',
 'Subject5_AccTempEDA.dat',
 'Subject16_AccTempEDA.dat',
 'Subject19_SpO2HR.dat',
 'Subject1_SpO2HR.dat',
 'Subject2_SpO2HR.hea',
 'Subject4_AccTempEDA.dat',
 'Subject17_SpO2HR.csv',
 'Subject20_SpO2HR.csv',
 'Subject11_AccTempEDA.dat',
 'Subject20_AccTempEDA.atr',
 'Subject17_SpO2HR.hea',
 'Subject14_SpO2HR.dat',
 'Subject20_SpO2HR.hea',
 'Subject3_AccTempEDA.dat',
 'subjectinfo.csv',
 'Subject2_SpO2HR.csv',
 'Subject19_AccTempEDA.dat',
 'Subject1_AccTempEDA.dat',
 'Subject5_SpO2HR.csv',
 'Subject13_AccTempEDA.dat',
 'Subject20_AccTempEDA.hea',
 'Subject8_SpO2HR.hea',
 'Subject10_SpO2HR.hea',
 'Subject13_SpO2HR.dat',
 'Subject9_AccTempEDA.dat',
 'ANNOTATORS',
 'Subject6_AccTemp

In [42]:
for file in os.listdir():
    if file.endswith('_SpO2HR.csv'):
        df = pd.read_csv(file)
        # print(df.head())
        df.drop(columns = ['time'], inplace=True)
        sync_df=pd.DataFrame(np.repeat(df.values, 8, axis=0), columns=df.columns)
        sync_df['time'] = np.arange(len(sync_df)) / 8
        sync_df.to_csv(file[:-4] + "_sync.csv", index=False)


In [62]:
for file in os.listdir():
    for i in range(1,21):
        if file.startswith("Subject" + str(i)) and file.endswith("_sync.csv"):
            # read file in
            sync_df = pd.read_csv(file)
            # add other dataframe to sync df
            for other_file in os.listdir():
                if other_file.startswith("Subject" + str(i)) and other_file.endswith("AccTempEDA.csv"):
                    acc_df = pd.read_csv(other_file)
                    # merge dataframes
                    merged_df = pd.merge(sync_df, acc_df, how='inner', on='time')
                    # add physiological stress column
                    # need the seconds for the physiological stress column -- each entry is 1/8 of a second, so need entries between 5 minutes and 10 minutes -- 5*60*8 for the start of the 5 minutes, then need 5 minutes after that so 10*60*8 is when the 5 minutes ends
                    merged_df['physiological_stress'] = np.where((merged_df['time'] >= 5*60*8) & (merged_df['time'] <= 10*60*8), 1, 0)
                    # save merged dataframe
                    merged_df.to_csv("Subject" + str(i) + "_merged.csv", index=False)

# now we have all the merged dataframes, need to split them into training, validation, and testing sets -- each subject is named subjectX_merged.csv

# train test splitting

## normalization

# missing data

# outlier detection

# sensor fusion

# discretization

# feature selection

# feature extraction